Survival analysis of CPTAC data (including code for Fig. 5D)

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
#import scipy.stats  as stats
pd.set_option('display.max_columns', None)
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter

In [ ]:
proteomics=pd.read_excel('mmc3.xlsx',sheet_name='proteome_normalized')
clinical=pd.read_excel('mmc2.xlsx',sheet_name='clinical_data')
mutations=pd.read_excel('mmc2.xlsx',sheet_name='additional_annotations')

In [ ]:
IDH_mutant_tumors = list(mutations[mutations['multiomic']=='IDH mutant']['case'])
clinical2=clinical[['case_id','age','gender','path_diag_to_last_contact_days','path_diag_to_death_days','lost_to_follow_up']]
protein_tumor_set=list(proteomics.columns[3:-10])
protein_tumor_set=list(set(protein_tumor_set).difference(IDH_mutant_tumors))
clinical2=clinical2[clinical2['case_id'].isin(protein_tumor_set)]
clinical2['Male']=1*(clinical2['gender']=='Male')
clinical2=clinical2.drop(['gender'],1)
proteomics2=proteomics[['symbol']+protein_tumor_set]

In [ ]:
def create_combined_df(survival_df,prot_list):
    proteomics_red=proteomics2[proteomics2['symbol'].isin(prot_list)]
    proteomics_red_T=proteomics_red.set_index('symbol').T
    proteomics_red_T['case_id']=proteomics_red_T.index
    proteomics_red_T=proteomics_red_T.reset_index().drop(['index'],1)
    df_merged=survival_df.merge(proteomics_red_T,how='outer',on='case_id')
    return df_merged

def get_time_status_for_cox(cur_df):
#cur_df=clinical_lo_E_hi_C
    tumor_time_status={}
    for row_num in range(len(cur_df)):
        this_row=cur_df.iloc[row_num]
        tumor=this_row['case_id']
        if np.isnan(this_row['path_diag_to_last_contact_days']) and np.isnan(this_row['path_diag_to_death_days']):
            continue
        else:
            if not np.isnan(this_row['path_diag_to_death_days']):
                last_obs=this_row['path_diag_to_death_days']
                status=1
            else:
                last_obs=this_row['path_diag_to_last_contact_days']
                status=0
        tumor_time_status[tumor]=[last_obs,status]
    new_df=pd.DataFrame.from_dict(tumor_time_status,orient='index').rename(columns={0:'last_observation',1:'status'})
    new_df['case_id']=new_df.index
    return new_df.reset_index(drop=True)

In [ ]:
survival_info_df=get_time_status_for_cox(clinical2).merge(clinical2[['case_id','age','Male']],how='left')
df_combined=create_combined_df(survival_info_df,['CD163'])
df_combined=df_combined.dropna()

In [ ]:
cph = CoxPHFitter()
kmf = KaplanMeierFitter()

In [ ]:
def clean_cph_table(this_df):
    return this_df[['exp(coef)','exp(coef) lower 95%','exp(coef) upper 95%','p']]
def get_survival_stats_sets(cur_set1,cur_set2):
    this_df=df_combined[df_combined['case_id'].isin(cur_set1+cur_set2)]
    this_df['in_group']=1*(this_df['case_id'].isin(cur_set1))
    cph.fit(this_df, duration_col='last_observation', event_col='status', formula="in_group+Male+age")
    return clean_cph_table(cph.summary)
def format_stats(tab):
    HR=tab['exp(coef)']['in_group']
    HR_lo=tab['exp(coef) lower 95%']['in_group']
    HR_hi=tab['exp(coef) upper 95%']['in_group']
    p_val=tab['p']['in_group']
    return 'Cox HR = %.2f'%HR+ ' [%.2f'%HR_lo+',%.2f'%HR_hi+'], p = %.3f'%p_val

def show_survival_curves(tumors1,name1,color1,tumors2,name2,color2,cox_stats,fname):
    df1=df_combined[df_combined['case_id'].isin(tumors1)]

    df2=df_combined[df_combined['case_id'].isin(tumors2)]
    

    fig, ax = plt.subplots(figsize=(12, 10))


    df_base = df1.dropna()
    df_base_name=name1
    df2 = df2.dropna()
    df2_name=name2

    lr_result2=logrank_test(df_base['last_observation'], df2['last_observation'], event_observed_A=df_base['status'], event_observed_B=df2['status'])
    pval2=lr_result2.p_value
    plt.rcParams["font.size"] = 30
    plt.rcParams["font.family"] = 'Arial'
    kmf.fit(df_base['last_observation'],df_base['status'],label=df_base_name).plot_survival_function(ax=ax,color=color1)
    kmf.fit(df2['last_observation'],df2['status'],label=df2_name).plot_survival_function(ax=ax,color=color2)
    plt.xlabel('Days',fontsize=30)
    plt.ylabel('Proportion Survived',fontsize=30)
    plt.legend()
    plt.title('Effect of CD163 protein expression on survival',fontsize=30)
    plt.text(-0.02,0.02,'Log-rank p ='+'{:.3f}'.format(pval2)+'\n'+cox_stats,fontsize=28)
    #plt.title('Log-rank p ='+'{:.3f}'.format(pval2))
    plt.setp(ax.artists, edgecolor = 'black')
    plt.setp(ax.lines, color='black')
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(1.2)
    plt.tight_layout()
    #plt.savefig(fname)
    plt.show()

In [ ]:
q_los=[1/3,1/4,1/5,1/2-0.001]
q_his=[2/3,3/4,4/5,1/2+0.001]
ind=0
q_lo=q_los[ind]
q_hi=q_his[ind]

prot='CD163'
quants=df_combined[prot].quantile([q_lo,q_hi])
lo_thresh=quants[q_lo]
hi_thresh=quants[q_hi]
lo_CD163_tumors=list(df_combined[df_combined[prot]<lo_thresh]['case_id'])
hi_CD163_tumors=list(df_combined[df_combined[prot]>=hi_thresh]['case_id'])
tab=get_survival_stats_sets(hi_CD163_tumors,lo_CD163_tumors)
c_stats=format_stats(tab)
t1=hi_CD163_tumors
n1='Expression in top 1/3'
c1='green'
t2=lo_CD163_tumors
n2='Expression in bottom 1/3'
c2='black'
show_survival_curves(t1,n1,c1,t2,n2,c2,c_stats,'CPTAC_top_bot_tertiles_IDH_mut_excluded.pdf')